In [0]:
!pip install tensorflow
# TensorFlow and tf.keras
# its just working with tensorflow 1.13.1, with others has problem


In [0]:
import tensorflow as tf
print(tf.__version__)

1.13.1


In [0]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!ls "/content/gdrive/My Drive/NLPHW1"
root_path = '/content/gdrive/My Drive/NLPHW1/'
print(root_path)

 130K_best_model		     'LastCode_NLPHW1 (2).ipynb'
 cityu_best_models		      msr_best_models
'Copy of LastCode_NLPHW1 (2).ipynb'   pku_best_models
 Datasets			      tf1.12.0TestPrediction.ipynb
 embedding
/content/gdrive/My Drive/NLPHW1/


**imports and basic definitions**

In [0]:
# Imports
import os
import re
import errno
import pickle
import numpy as np
from collections import Counter
import codecs
import tensorflow as tf
from tensorflow.contrib import layers
from tensorflow.contrib import crf

#!pip install HanziConv
# from hanziconv import HanziConv

# Added Chars to dictionary
START_SENT = "گ"
END_SENT = "چ"
UNK = "ژ"
PAD = "پ"
TAGB,TAGI,TAGE,TAGS=0,1,2,3

rNUM = '(-|\+)?\d+((\.|·)\d+)?%?'
rENG = '[A-Za-z_.]+'

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise


***Create Curpus Section For Cleaning and combining datasets***

In [0]:
# ===-----------------------------------------------------------------------===
# CreateCurpus
# ===-----------------------------------------------------------------------===

# Function for converting traditional chinease to simplefied one.
def traditional_to_siplefied(src_file, des_file, encode='UTF-8'):
   with open(src_file, encoding=encode) as src_file, open(des_file, 'w', encoding=encode) as des_file:
       for line in src_file:                    
           rstring = HanziConv.toSimplified(line)                        
           des_file.write(rstring)

# function for combining datasets as one large set.
def combine_files(files, out):
    if os.path.exists(out):
        os.remove(out)
    with open(out, 'a', encoding='utf-8') as out:
        for file_num in range(len(files)): 
            file_add = root_path+'Datasets/{}.utf8'.format(files[file_num])
            print(file_add)
            with open(file_add, encoding='utf-8') as file_add:
                for line in file_add:
                    out.write(line) 
            if(file_num != len(files)-1): 
                print("new line")
                out.write('\n')    
    print('\nAll data Saved as All_datasets file')
                


# Function for cleaning file, make charachters better and spaces standard space.
def clean_data(src_file, des_file, encode='UTF-8'):
    with open(src_file, encoding=encode) as src_file, open(des_file, 'w', encoding=encode) as des_file:
        sentences=[]
        for line in src_file: 
            # cleaning input
            rstring = ""
            for uchar in line:
                inside_code = ord(uchar)
                if inside_code == 12288:
                    inside_code = 32
                elif 65281 <= inside_code <= 65374:
                    inside_code -= 65248
            
                rstring += chr(inside_code)      
            
            sent_words = rstring.split()
            
            new_sent_words = []
            
            for word in sent_words:
                word = re.sub('\s+', '', word, flags=re.U)    
                new_sent_words.append(word)
            
            sentences.append(new_sent_words)
        for sent_num in range(len(sentences)):
           # if(sent_num != len(sentences)-1):
            des_file.write(' '.join(sentences[sent_num]) + '\n') 
            #else:
             #   des_file.write(' '.join(sentences[sent_num])) 


def cleaning_datasets(all_datasets,with_space =True):    
    make_sure_path_exists(root_path+'Datasets/cleaned_data/{}'.format(all_datasets))
    make_sure_path_exists(root_path+'Datasets/cleaned_data/{}'.format(all_datasets) + '/raw')
    clean_data(root_path+'Datasets/training/{}_training.utf8'.format(all_datasets), root_path+'Datasets/cleaned_data/{}/raw/train.utf8'.format(all_datasets))
    clean_data(root_path+'Datasets/gold/{}_test_gold.utf8'.format(all_datasets), root_path+'Datasets/cleaned_data/{}/raw/test.utf8'.format(all_datasets))   

def remove_spaces(src_file, des_file, encode='UTF-8'):
    with open(src_file, encoding=encode) as src_file, open(des_file, 'w', encoding=encode) as des_file:
        for line in src_file: 
            # cleaning input                     
            sent_words = line.split()                             
            sentence = ''.join(sent_words) + '\n'
            des_file.write(sentence)     


# function for creating BIES tag
def bies_tag(input_file, output_file):
    with open(input_file, encoding='utf-8') as input_data, open(output_file, 'w', encoding='utf-8') as output_data:
        for line in input_data:
            word_list = line.strip().split()
            tag_list = ''
            for word in word_list:
                if len(word) == 1:
                    tag_list +='S'
                else:
                    tag_list+='B'
                    for w in word[1:len(word) - 1]:
                        tag_list += 'I'
                    tag_list += 'E'            
            tag_list += '\n'
            #print(tag_list)
            #print(line)
            output_data.write(tag_list)
            
def make_bies(all_datasets):    
    make_sure_path_exists(root_path+'Datasets/cleaned_data/{}'.format(all_datasets))
    make_sure_path_exists(root_path+'Datasets/cleaned_data/{}'.format(all_datasets) + '/bies')
    bies_tag(root_path+'Datasets/training/{}_training.utf8'.format(all_datasets), root_path+'Datasets/cleaned_data/{}/bies/train.utf8'.format(all_datasets))
    bies_tag(root_path+'Datasets/gold/{}_test_gold.utf8'.format(all_datasets), root_path+'Datasets/cleaned_data/{}/bies/test.utf8'.format(all_datasets))



In [0]:
# create curpus from training and gold dataset files, it gives us one file 
#containing all training sets in single file and all gold_test set files in one another file
print('create_curpus started')    

#*** traditional to simplefied converting ====================================

#traditional_to_siplefied('Datasets/training/as_training.utf8','Datasets/training/simple_as_training.utf8')
#traditional_to_siplefied('Datasets/training/cityu_training.utf8','Datasets/training/simple_cityu_training.utf8')
#traditional_to_siplefied('Datasets/gold/cityu_test_gold.utf8','Datasets/gold/simple_cityu_test_gold.utf8')
#traditional_to_siplefied('Datasets/gold/as_test_gold.utf8','Datasets/gold/simple_as_test_gold.utf8')


#*** combining all dataset in one file =====================================

train_files = ['training/pku_training','training/msr_training','training/simple_cityu_training','training/simple_as_training']
combine_files(train_files ,root_path+'Datasets/training/All_datasets_training.utf8')        

gold_files = ['gold/pku_test_gold','gold/msr_test_gold','gold/simple_cityu_test_gold','gold/simple_as_test_gold']
combine_files(gold_files ,root_path+'Datasets/gold/All_datasets_test_gold.utf8')

# End ==========================================================================================================


create_curpus started
/content/gdrive/My Drive/NLPHW1/Datasets/training/pku_training.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/training/msr_training.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/training/simple_cityu_training.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/training/simple_as_training.utf8

All data Saved as All_datasets file
/content/gdrive/My Drive/NLPHW1/Datasets/gold/pku_test_gold.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/gold/msr_test_gold.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/gold/simple_cityu_test_gold.utf8
new line
/content/gdrive/My Drive/NLPHW1/Datasets/gold/simple_as_test_gold.utf8

All data Saved as All_datasets file


In [0]:
#*** some cleaning in dataset and remove spaces if there is any ===================
# All data Toghether for making Embedding and ngram2id files.
cleaning_datasets('All_datasets')
cleaning_datasets('pku')
cleaning_datasets('msr')
cleaning_datasets('simple_cityu')
cleaning_datasets('simple_as')



In [0]:
#*** making bies file ===================    
make_bies('All_datasets')  
make_bies('pku')    
make_bies('msr') 
make_bies('simple_cityu') 
make_bies('simple_as') 
#*** remove space from data =================== 
#remove_spaces('Datasets/All_datasets/raw/train.utf8', 'Datasets/All_datasets/raw/no_space_train.utf8')
#remove_spaces('Datasets/All_datasets/raw/test.utf8', 'Datasets/All_datasets/raw/no_space_test.utf8')

print('create_curpus all finished')    


create_curpus all finished


***Preprocessing Section For creating BIES files and Making Dataset Ready For Tensorflow Model***


```
# pkl file of all data created and saved for any further use of data.
```



In [0]:
# ==========================================================================================================
# PreProcessing
# ==========================================================================================================

#get dictionary from characters and bigrams and trigrams to id    
def get_ngram2id(filename,bigram_words,trigram_words,fourgram_words,m_f_bigram,m_f_3gram,m_f_4gram):
    # Adding some chars that we define as start sentence, end sentence, padding and UNK.
    x=[UNK,PAD,START_SENT,END_SENT]
    # gets unigrams from dataset
    with codecs.open(filename,'r','utf-8') as f:        
        for line in f:
            line = line.strip().split()
            for uchar in line:                
                x.extend(uchar)
    # gets bigrams from saved bigrams of dataset
    bigrams=[]
    if bigram_words is not None:
        with codecs.open(bigram_words,'r','utf-8') as f:
            for line in f:
                com=line.strip().split()
                if int(com[1])>m_f_bigram:
                    bigrams.append(com[0])
    x.extend(bigrams)
    
    # gets trigrams from saved trigrams of dataset
    trigram=[]
    if trigram_words is not None:
        with codecs.open(trigram_words,'r','utf-8') as f:
            for line in f:
                com=line.strip().split()
                if int(com[1])>m_f_3gram:                
                    trigram.append(com[0])
    x.extend(trigram)
    # gets fourgrams from saved trigrams of dataset
    fourgram=[]
    if fourgram_words is not None:
        with codecs.open(fourgram_words,'r','utf-8') as f:
            for line in f:
                com=line.strip().split() 
                if int(com[1])>m_f_4gram:
                    fourgram.append(com[0])
    x.extend(fourgram)
    
    
    counter = Counter(x)    
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))
    words, _ = list(zip(*count_pairs))
    ngram2id = dict(zip(words, range(len(words))))
    return ngram2id

#convert words to tag_id for training set labels
def word2tag_id(word):
    if len(word)==1:
        return [TAGS]
    if len(word)==2:
        return [TAGB,TAGE]
    tag_id=[]
    tag_id.append(TAGB)
    for i in range(1,len(word)-1):
        tag_id.append(TAGI)
    tag_id.append(TAGE)
    return tag_id

# getting ngrams function
def ngram(ustr,n=2):
    ngram_list=[]
    for i in range(len(ustr)-n+1):
        ngram_list.append(ustr[i:i+n])
    return ngram_list

# getting and saving ngrams function as pkl file
def save_ngram_words(src_dataset,dst_dataset,window_size=2):
    words=dict()
    with codecs.open(src_dataset,'r','utf-8') as f:
        for line in f:
            line=re.sub('\s+','',line.strip())
            for word in ngram(line,window_size):
                words[word]=words.get(word,0)+1
    with codecs.open(dst_dataset,'w','utf-8') as f:
        len(words)
        i = 1
        for k,v in words.items():             
            if(i!=len(words)):
                f.write(k+' '+str(v)+'\n')
                i = i+1
            else:
                f.write(k+' '+str(v))

# getting training data function with unigram, bigram, trigrams and fourgram (list of 14 data for every character)
def get_train_data(filename,ngrams2id, usebigram, usetrigram,usefourgram):   
    x,y=[],[]
    with codecs.open(filename,'r','utf-8') as f:
        # for every line we split and then tag every word with proper id.
        for line in f:
            word_list=line.strip().split()
            line_y=[]
            line_x=[]
            for word in word_list:
                line_y.extend(word2tag_id(word))
            y.append(line_y)
            # for every line we remove spaces and then assume every 5 (2 left char and 2 right char)
            # char as a new context, for every context we also calculate bigram and trigram and add them too
            line=re.sub(u'\s+','',line.strip())
            contexs=window(line)
            for contex in contexs:
                charx=[]
                #contex window
                charx.extend([ngrams2id.get(c,ngrams2id[UNK]) for c in contex])
                #bigram feature
                if usebigram:
                    charx.extend([ngrams2id.get(bigram, ngrams2id[UNK]) for bigram in ngram(contex,2)])
                #bigram feature
                if usetrigram:
                    charx.extend([ngrams2id.get(trigram, ngrams2id[UNK]) for trigram in ngram(contex,3)])
                #bigram feature
                if usefourgram:
                    charx.extend([ngrams2id.get(fourgram, ngrams2id[UNK]) for fourgram in ngram(contex,4)])
                line_x.append(charx)
            x.append(line_x)
            assert len(line_x)==len(line_y)
    return x,y

# window return sentence as list of 5 length context, for every word we are 
# gettnig 4 neighbour, 2 left and 2 right for having more training data.
def window(ustr,left=2,right=2):
    sent=''
    for i in range(left):
        sent+=START_SENT
    sent+=ustr
    for i in range(right):
        sent+=END_SENT
    windows=[]
    for i in range(len(ustr)):
        windows.append(sent[i:i+left+right+1])
    return windows



In [0]:
# get ngrams2id and id2ngram
FILE_FOR_NGRAM = 'All_datasets'
ALL_TRAIN_FILENAME=root_path+'Datasets/cleaned_data/{}/raw/train.utf8'.format(FILE_FOR_NGRAM)
TEST_FILENAME=root_path+'Datasets/cleaned_data/{}/raw/test.utf8'.format(FILE_FOR_NGRAM)
BIGRAM_FILENAME = root_path+'Datasets/cleaned_data/{}/raw/train_bigram.utf8'.format(FILE_FOR_NGRAM)
TRIGRAM_FILENAME = root_path+'Datasets/cleaned_data/{}/raw/train_trigram.utf8'.format(FILE_FOR_NGRAM)
FOURGRAM_FILENAME = root_path+'Datasets/cleaned_data/{}/raw/train_fourgram.utf8'.format(FILE_FOR_NGRAM)

In [0]:
# bigrams 
save_ngram_words(ALL_TRAIN_FILENAME, BIGRAM_FILENAME, 2)

# Trigram
save_ngram_words(ALL_TRAIN_FILENAME, TRIGRAM_FILENAME, 3)

# fourgram
save_ngram_words(ALL_TRAIN_FILENAME, FOURGRAM_FILENAME, 4)


In [0]:
m_f_4gram = 8
m_f_3gram = 3
m_f_bigram = 0
ngrams2id = get_ngram2id(ALL_TRAIN_FILENAME, BIGRAM_FILENAME, TRIGRAM_FILENAME,FOURGRAM_FILENAME,m_f_bigram,m_f_3gram,m_f_4gram)
PAD_ID = ngrams2id[PAD]

In [0]:
print(len(ngrams2id))
print(PAD_ID)


1773043
69977


In [0]:
##saving All data ngrams2id as pkl file.
make_sure_path_exists(os.path.dirname(root_path+'Datasets/output_{}_ngram2id750K'.format(FILE_FOR_NGRAM)))
print('Saving dataset to {}'.format(root_path+'Datasets/output_{}_ngram2id750K'.format(FILE_FOR_NGRAM)))  

output_ngram2id = {}    
output_ngram2id["ngrams2id"] = ngrams2id

with open(root_path+'Datasets/output_{}_ngram2id750K.pkl'.format(FILE_FOR_NGRAM), "wb") as outfile:
    pickle.dump(output_ngram2id, outfile)



Saving dataset to /content/gdrive/My Drive/NLPHW1/Datasets/output_All_datasets_ngram2id750K


In [0]:
# Because of the limitation of colab I decided to train model sequentially for each dataset
# I load each dataaset in different time and train model then save that and restart colab session
# and load another dataset and model and train that and ... .

def separate_datasets_for_train(file_name):
    TRAIN_FILENAME=root_path+'Datasets/cleaned_data/{}/raw/train.utf8'.format(file_name)   
    TEST_FILENAME=root_path+'Datasets/cleaned_data/{}/raw/test.utf8'.format(file_name)
    
    # getting the train, test and dev data in proper format.
    train_x, train_y = get_train_data(TRAIN_FILENAME,ngrams2id, usebigram=True, usetrigram = True,usefourgram = True)
    test_x, test_y = get_train_data(TEST_FILENAME, ngrams2id, usebigram=True, usetrigram = True,usefourgram = True)

    # Train and Dev split
    from sklearn.model_selection import train_test_split
    train_x, dev_x, train_y, dev_y = train_test_split(train_x, train_y, test_size=0.05, random_state=42)

    print("size of train data ....")
    print(len(train_x))
    print(len(train_x[0]))
    print(len(train_x[0][0]))
    print("size of dev data ....")
    print(len(dev_x))
    print(len(dev_x[0]))
    print(len(dev_x[0][0]))
    print("size of test data ....")
    print(len(test_x))
    print(len(test_x[0]))
    print(len(test_x[0][0]))        
    
    return train_x, train_y, dev_x, dev_y, test_x, test_y
  
def save_pkl_datasets(file_name):
  # saving All data as pkl file.
    SAVED_ADD = root_path+'Datasets/output_{}_train_test'.format(file_name)    
    make_sure_path_exists(os.path.dirname(SAVED_ADD))
    print('Saving dataset to {}'.format(SAVED_ADD))    

    output_train_test = {}    

    output_train_test["train_x"] = train_x
    output_train_test["train_y"] = train_y
    output_train_test["dev_x"] = dev_x
    output_train_test["dev_y"] = dev_y
    output_train_test["test_x"] = test_x
    output_train_test["test_y"] = test_y
    
    with open(root_path+'Datasets/output_{}_train_test.pkl'.format(file_name), "wb") as outfile:
        pickle.dump(output_train_test, outfile)
    

In [0]:
# create train, test and dev sets and saving them as pkl file for further loading and making subset from them.

datasetname = 'pku' # 'msr' 'simple_cityu' 'simple_as' 'All_datasets'
train_x, train_y, dev_x, dev_y, test_x, test_y = separate_datasets_for_train(datasetname)

save_pkl_datasets(datasetname)

subset_train = 30000 # Put number of subset data you want for train
subset_dev= 1000
subset_test= 1000

train_x = train_x#[slice(0, subset_train)]
train_y = train_y#[slice(0, subset_train)]  
dev_x = dev_x#[slice(0, subset_dev)]
dev_y = dev_y#[slice(0, subset_dev)]
test_x = test_x#[slice(0, subset_test)]
test_y = test_y#[slice(0, subset_test)]


size of train data ....
18103
19
14
size of dev data ....
953
150
14
size of test data ....
1945
21
14


In [0]:
# code for getting concatenation of subsets of all 4 datasets.

train_x, train_y, dev_x, dev_y, test_x, test_y = [],[],[],[],[],[]

train_x1, train_y1, dev_x1, dev_y1, test_x1, test_y1 = separate_datasets_for_train('pku')

# Adding All pku 18000
train_x += train_x1
train_y += train_y1
dev_x += dev_x1
dev_y += dev_y1
test_x += test_x1
test_y += test_y1

# adding 30000 of msr
train_x1, train_y1, dev_x1, dev_y1, test_x1, test_y1 = separate_datasets_for_train('msr')

subset_train = 30000 # Put number of subset data you want for train
subset_dev= 1000

train_x1 = train_x1[slice(0, subset_train)]
train_y1 = train_y1[slice(0, subset_train)]  
dev_x1 = dev_x1[slice(0, subset_dev)]
dev_y1 = dev_y1[slice(0, subset_dev)]

train_x += train_x1
train_y += train_y1
dev_x += dev_x1
dev_y += dev_y1
test_x += test_x1
test_y += test_y1

# adding 30000 of simple_cityu
train_x1, train_y1, dev_x1, dev_y1, test_x1, test_y1 = separate_datasets_for_train('simple_cityu')

subset_train = 30000 # Put number of subset data you want for train
subset_dev= 1000

train_x1 = train_x1[slice(0, subset_train)]
train_y1 = train_y1[slice(0, subset_train)]  
dev_x1 = dev_x1[slice(0, subset_dev)]
dev_y1 = dev_y1[slice(0, subset_dev)]
  
train_x += train_x1
train_y += train_y1
dev_x += dev_x1
dev_y += dev_y1
test_x += test_x1
test_y += test_y1


# Adding 50000 of simple_as
train_x1, train_y1, dev_x1, dev_y1, test_x1, test_y1 = separate_datasets_for_train('simple_as')

subset_train = 50000 # Put number of subset data you want for train
subset_dev= 1000

train_x1 = train_x1[slice(0, subset_train)]
train_y1 = train_y1[slice(0, subset_train)]  
dev_x1 = dev_x1[slice(0, subset_dev)]
dev_y1 = dev_y1[slice(0, subset_dev)]
  
train_x += train_x1
train_y += train_y1
dev_x += dev_x1
dev_y += dev_y1
test_x += test_x1
test_y += test_y1


size of train data ....
50368
7
14
size of dev data ....
2651
38
14
size of test data ....
1493
15
14


In [0]:
# overall for last model I'm using concatenation of 130K data fror train and for better results 
# I shuffled train data for getting good distribution of all 4 datasets
import random

mapIndexPosition = list(zip(train_x, train_y))
random.shuffle(mapIndexPosition)
train_x, train_y = zip(*mapIndexPosition)


In [0]:
print("size of train data ....")
print(len(train_x))
print(len(train_x[0]))
print(len(train_x[0][0]))
print("size of dev data ....")
print(len(dev_x))
print(len(dev_x[0]))
print(len(dev_x[0][0]))
print("size of test data ....")
print(len(test_x))
print(len(test_x[0]))
print(len(test_x[0][0]))        

# save_pkl_datasets('concat_130k')

size of train data ....
82577
19
14
size of dev data ....
4347
43
14
size of test data ....
3985
13
14


**Training Basec Functions**


*   Defning Padding Functions
*   Defining Functions for Number of Hits in Every Batch
*   Retriev All Needed Data From PKL File.
*   Train, Dev Split With sklearn traing_test_split function with Shuffle






In [0]:
# Retriev word_embeddings and PAD word id from Saved pkl File
FILE_FOR_NGRAM = 'All_datasets'
ngrams2id = pickle.load(open(root_path+'Datasets/output_{}_ngram2id750K.pkl'.format(FILE_FOR_NGRAM), "rb"))["ngrams2id"]
print(len(ngrams2id))

743480


In [0]:
# use pretrained embeddings function, for chars we are using pretrain file and for bigram, trigram 
# and fourgram we are using mean of embeddings of all unigrams of them

def reading_pretrained_embeddings(filename, ngrams2id):
    # Reading Pretrained Embeddings from file
    pretrain_embeddigs = {}
    with codecs.open(filename, "r", "utf-8") as f:
        for line in f:
            pre_train = line.split()
            if len(pre_train) > 2:
                word = pre_train[0]
                if word in ngrams2id:
                    vec = pre_train[1:]
                    pretrain_embeddigs[word] = vec                
    
    print("pretrained embedding files reading finished ...")
    # making embeddings for all ngrams2id.
    embedding_dim = len(next(iter(pretrain_embeddigs.values())))
    out_of_vocab = 0
    out = np.ones((len(ngrams2id), embedding_dim))
    for ngram in ngrams2id.keys():
        if len(ngram) == 1:
            if ngram in pretrain_embeddigs.keys():        
                out[ngrams2id[ngram]]=np.array(pretrain_embeddigs[ngram])
            else:
                out_of_vocab+=1
                np.random.uniform(-0.5, 0.5, embedding_dim)
    for ngram in ngrams2id.keys():        
        #embedding for bigrams
        if len(ngram) == 2:            
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]])/2
        #embedding for trigrams
        if len(ngram) == 3:
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]]+out[ngrams2id[ngram[2]]])/3
        #embedding for fourgrams
        if len(ngram) == 4:
            out[ngrams2id[ngram]]= (out[ngrams2id[ngram[0]]]+out[ngrams2id[ngram[1]]]+out[ngrams2id[ngram[2]]]+out[ngrams2id[ngram[3]]])/4               
    print('out_of_vocab characters: %d' % (out_of_vocab) )         
    return out,out_of_vocab


In [0]:
# get ngrams2ids's pretrained embeddings
word_embeddings, out_of_vocab = reading_pretrained_embeddings(root_path+'embedding/character.vec', ngrams2id)
PAD_ID = ngrams2id[PAD]


pretrained embedding files reading finished ...
out_of_vocab characters: 380


In [0]:
print(PAD_ID)
print(len(word_embeddings))

69977
1773043


In [0]:
# #Retriev train, test data from pkl files

# FILE_NAME = 'pku'

# TRAIN_FILE_ADD = root_path+'Datasets/output_{}_train_test.pkl'.format(FILE_NAME)

# subset_train = 1000 # Put number of subset data you want for train
# subset_dev= 100
# subset_test= 1000

# train_x = pickle.load(open(TRAIN_FILE_ADD, "rb"))["train_x"][slice(0, subset_train)]
# train_y = pickle.load(open(TRAIN_FILE_ADD, "rb"))["train_y"][slice(0, subset_train)]  
# dev_x = pickle.load(open(TRAIN_FILE_ADD, "rb"))["dev_x"][slice(0, subset_dev)]
# dev_y = pickle.load(open(TRAIN_FILE_ADD, "rb"))["dev_y"][slice(0, subset_dev)]
# test_x = pickle.load(open(TRAIN_FILE_ADD, "rb"))["test_x"][slice(0, subset_test)]
# test_y = pickle.load(open(TRAIN_FILE_ADD, "rb"))["test_y"][slice(0, subset_test)]

# print(len(train_x))
# print(len(dev_x))
# print(len(test_x))

1000
100
1000


In [0]:
# #Retriev All train, test data from pkl files
# FILE_NAME = ['pku','msr','simple_cityu','simple_as']

# train_x, train_y, dev_x, dev_y, test_x, test_y = [],[],[],[],[],[]

# for file in FILE_NAME:
#     TRAIN_FILE_ADD = root_path+'Datasets/output_{}_train_test.pkl'.format(file)
#     train_test_file = pickle.load(open(TRAIN_FILE_ADD, "rb"))
#     train_x += train_test_file["train_x"]
#     train_y += train_test_file["train_y"]
#     dev_x += train_test_file["dev_x"]
#     dev_y += train_test_file["dev_y"]
#     test_x += train_test_file["test_x"]
#     test_y += train_test_file["test_y"]
   
# print(len(train_x))
# print(len(train_x[0]))
# print(len(train_x[0][0]))

# print(len(dev_x))
# print(len(dev_x[0]))
# print(len(dev_x[0][0]))

# print(len(test_x))
# print(len(test_x[0]))
# print(len(test_x[0][0]))



**TensorFlow Model Creating**

*   pre_trained Embedding is used.
*   Bidirectional LSTM with Stacked RNN Cells is Used.
*   Loss is negative log_likelihood and gradiant.



In [0]:
# ===-----------------------------------------------------------------------===
# Trainig Section
# ===-----------------------------------------------------------------------===

# function for add padding for train_y in every batch based on maximum length of sentence in that batch
# this method boost the performance of running.
def padding(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X

# padding function for train_x
def padding3(X,padding_word):
	max_len = 0
	for x in X:
		if len(x) > max_len:
			max_len = len(x)
	padded_X = np.ones((len(X), max_len,14), dtype=np.int32) * padding_word
	for i in range(len(X)):
		for j in range(len(X[i])):
			padded_X[i, j] = X[i][j]
	return padded_X

# function for calculating umber of hits and number of all predictions.
def number_of_batch_hits(y_batch_pred,y_batch_true):
    num_hits = 0
    num_all_chars = 0    
    for i in range(len(y_batch_true)):
        for j in range(len(y_batch_true[i])):
            num_all_chars = num_all_chars+1
            if y_batch_pred[i][j] == y_batch_true[i][j]:
                num_hits = num_hits+1    
    return num_hits, num_all_chars

# ----------------- Add Summary Function ------------------------------------------
def add_summary(writer, name, value, global_step):
    summary = tf.Summary(value=[tf.Summary.Value(tag=name, simple_value=value)])
    writer.add_summary(summary, global_step=global_step)


In [0]:
# we are using pretrained embedding
VOCAB_SIZE =  word_embeddings.shape[0]
WORD_EMBEDDING_DIM = word_embeddings.shape[1]
print(WORD_EMBEDDING_DIM)
print(VOCAB_SIZE)

# some Basic Hyper parameters
NUM_EPOCHS = 20
BATCH_SIZE = 16
HIDDEN_LAYER_DIM = 128
LEARNING_RATE = 0.001
NUM_CLASSES = 4
L2_REGU_LAMBDA=0.0001
NUM_LAYERS = 1
CLIP=5

100
1773043


In [0]:
# --------------------- Tensorflow part ---------------------------------------------------

def create_tensorflow_model(vocab_size, embedding_size, hidden_layer_dim):
    print("Creating TENSORFLOW model")
     
    # Inputs have (batch_size, timesteps) shape.
    X = tf.placeholder(dtype=tf.int32,shape=[None,None,14],name='input_x')   
    # Labels have (batch_size,) shape.
    labels = tf.placeholder(dtype=tf.int32,shape=[None,None],name='input_y')
    # dropout_keep_prob is a scalar.
    dropout_keep_prob=tf.placeholder(dtype=tf.float32,name='dropout_keep_prob')
    # Calculate sequence lengths to mask out the paddings later on.
    seq_length = tf.reduce_sum(tf.cast( tf.not_equal(X[:,:,2], tf.ones_like(X[:,:,2]) * PAD_ID), tf.int32), 1)
        
    with tf.variable_scope('embeddings', reuse=tf.AUTO_REUSE):
        embedding_matrix = tf.Variable(word_embeddings, dtype=tf.float32, name='embedding')
        #embedding_matrix = tf.get_variable("embeddings", shape=[vocab_size, embedding_size])
        embeddings = tf.nn.embedding_lookup(embedding_matrix, X)
        embeddings = tf.reshape(embeddings,[tf.size(X[:,1,1]),-1,14*embedding_size])
    
    # embeddings shape (batch size, sentence length with padding, 1400)
    embeddings=tf.nn.dropout(tf.cast(embeddings,tf.float32),dropout_keep_prob)
    
    with tf.variable_scope('rnn_cell', reuse=tf.AUTO_REUSE):
            print ('rnn_cell is lstm')
            def lstm_cell():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)
        
            def gru_cell():
                return tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(hidden_layer_dim), output_keep_prob=dropout_keep_prob)
        
            stacked_fw_lstm = tf.nn.rnn_cell.MultiRNNCell(
                [gru_cell() for _ in range(NUM_LAYERS)])
            
            stacked_bw_lstm = tf.nn.rnn_cell.MultiRNNCell(
                [lstm_cell() for _ in range(NUM_LAYERS)])
        
    with tf.variable_scope('rnn', reuse=tf.AUTO_REUSE):                        
            (forward_output,backword_output),_=tf.nn.bidirectional_dynamic_rnn(
                cell_fw=stacked_fw_lstm,
                cell_bw=stacked_bw_lstm,
                inputs=embeddings,
                sequence_length=seq_length,
                dtype=tf.float32
            )
            outputBidirection=tf.concat([forward_output,backword_output],axis=2)
            print ('outputBidirection is ok')    
                        
    print ('Loss Starts ....')
    with tf.variable_scope('loss', reuse=tf.AUTO_REUSE):
        output=layers.fully_connected(
            inputs=outputBidirection,
            num_outputs=NUM_CLASSES,
            activation_fn=None            
            )
        print ('output is ok ....')
        #crf
        log_likelihood, transition_params = crf.crf_log_likelihood(
            output, labels, seq_length)
        print ('crf_log_likelihood is ok ....')
        
        loss = tf.reduce_mean(-log_likelihood)
        print ('loss crf_log_likelihood is ok ....')
    
    print ('train-op Starts ....')
    with tf.variable_scope('train_op', reuse=tf.AUTO_REUSE):
        optimizer=tf.train.AdamOptimizer(learning_rate=LEARNING_RATE)
        #optimizer=tf.train.GradientDescentOptimizer(learning_rate=LEARNING_RATE, use_locking=False, name='GradientDescent') 
        print ('optimizer is ok ....')
    
        tvars=tf.trainable_variables()
        print ('tvars is ok ....')
    
        l2_loss = tf.add_n([tf.nn.l2_loss(v) for v in tvars if v.get_shape().ndims > 1])
        print ('l2_loss is ok ....')
        
        loss=loss+L2_REGU_LAMBDA*l2_loss
        print ('loss L2 is ok ....')
        
        grads,_=tf.clip_by_global_norm(tf.gradients(loss,tvars),CLIP)
        print ('grads is ok ....')
        
        train_op=optimizer.apply_gradients(zip(grads,tvars))
        print ('train_op apply_gradients is ok ....')
               
    return X, labels, output, train_op, dropout_keep_prob, loss, transition_params, seq_length


In [0]:
# create tensorflow model
X, labels, output, train_op, dropout_keep_prob, loss, transition_params, seq_length = create_tensorflow_model(VOCAB_SIZE, WORD_EMBEDDING_DIM, HIDDEN_LAYER_DIM)


Creating TENSORFLOW model
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
rnn_cell is lstm
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use tf.cast instead.
outputBidirection is ok
Loss Starts ....
output is ok ....
crf_log_likelihood is ok ....
loss crf_log_likelihood is ok ....
train-op Starts ....
optimizer is ok ....
tvars is ok ....
l2_loss is ok ....
loss L2 is ok ....


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


grads is ok ....
train_op apply_gradients is ok ....


***TensorFlow Model Training Section***

*   Batch Base Padding Is Performed
* 



In [0]:
# SUBSET_MODEL_ADD = root_path+'best_model_path68.5505176271315/model.ckpt'



In [0]:
# Run tensorflow model
saver = tf.train.Saver()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess: 
    # I couln't use tensorboard because it causes my colab session to crash for big datasets.
    #train_writer = tf.summary.FileWriter(root_path+'logging/tensorflow_model', sess.graph)

    sess.run(tf.global_variables_initializer())             
    print ('start session is ok ....')
    
    for epoch in range(NUM_EPOCHS):        
        #train
        train_loss=[]    
        for i in range(0, len(train_x), BATCH_SIZE):
            # slice dataset for padding
            batch_x = train_x[slice(i, i + BATCH_SIZE)]
            batch_y = train_y[slice(i, i + BATCH_SIZE)]
            batch_x = padding3(batch_x, PAD_ID)
            batch_y = padding(batch_y, PAD_ID)
            
            # runnig training session ang getting train loss for every batch and append in epoch loss
            _,loss_other=sess.run(
                [train_op, loss], feed_dict = { X:batch_x, labels:batch_y, dropout_keep_prob:0.3 })
            
            train_loss.append(loss_other)            
           # print ('Train batch %d loss %f' % (i, loss_other))
        train_loss=np.mean(train_loss,dtype=float)                
        print ('Train Epoch %d loss %f' % (epoch+1, train_loss))        
        print('--------------------------------') 
        
                                       
        #Dev
        dev_loss=[]
        dev_pred=[]
        dev_hit_all=0
        dev_all_char=0        
        for i in range(0, len(dev_x), BATCH_SIZE):
            batch_x = dev_x[slice(i, i + BATCH_SIZE)]
            batch_y = dev_y[slice(i, i + BATCH_SIZE)]
            batch_x = padding3(batch_x, PAD_ID)
            batch_y = padding(batch_y, PAD_ID)
            
            # lengths is length of every sentence(number of words), unary_scores is sentences with padding and
            # transition_param_dev is probabilities matrix
            loss_other,lengths,unary_scores,transition_param_dev=sess.run(
                [loss,seq_length,output, transition_params], feed_dict = { X:batch_x, labels:batch_y, dropout_keep_prob:1.0 })
            
            # predict is every batches sentences prediction and for preventing any empty line causing error in prediction
            # I use lenghs>0 condition and then append prediction to epoch prediction
            predict=[]
            for unary_score,length in zip(unary_scores,lengths):    
                if length > 0:
                    viterbi_sequence, _= tf.contrib.crf.viterbi_decode(unary_score[:length],transition_param_dev)                             
                    predict.append(viterbi_sequence)
                else:
                    predict.append("")
            
            dev_pred+=predict
            
            # number_of_batch_hits gave us all charachters of the batch and true predictions and we append
            #them to epoch hit and epoch all chars
            dev_batch_hit,dev_batch_char = number_of_batch_hits(predict,dev_y[slice(i, i + BATCH_SIZE)])            
            print('dev batch %d loss %f dev_batch_hit:%d dev_batch_char:%d batch_accuracy:%f' % (i, loss_other, dev_batch_hit, dev_batch_char,(dev_batch_hit/dev_batch_char)*100 ))
            dev_loss.append(loss_other)            
            dev_hit_all+=dev_batch_hit
            dev_all_char+=dev_batch_char
                
        dev_loss=np.mean(dev_loss,dtype=float)
        dev_acc = (dev_hit_all/dev_all_char)*100
        print('Valid Epoch %d loss %f' % (epoch+1,dev_loss))        
        print('dev_hit_all:%d dev_all_char:%d accuracy:%f' % (dev_hit_all,dev_all_char,dev_acc ))
        print('--------------------------------')  
        
        #add_summary(train_writer, "epoch_dev_loss", dev_loss, epoch+1)
        #add_summary(train_writer, "epoch_dev_acc", dev_acc, epoch+1)
        
        # Test        
        test_hit_all=0
        test_all_char=0        
        test_pred = []
        for i in range(0, len(test_x), BATCH_SIZE):
            batch_x = test_x[slice(i, i + BATCH_SIZE)]
            batch_y = test_y[slice(i, i + BATCH_SIZE)]
            batch_x = padding3(batch_x, PAD_ID)
            batch_y = padding(batch_y,PAD_ID)
            
            lengths,unary_scores,transition_param_test = sess.run(
                [seq_length,output,transition_params], feed_dict = {X:batch_x, dropout_keep_prob:1.0})
            predict=[]
            for unary_score,length in zip(unary_scores,lengths):
                if length > 0 :
                    viterbi_sequence, _=crf.viterbi_decode(unary_score[:length],transition_param_test)
                    predict.append(viterbi_sequence)
                else:
                    predict.append("")
                #print('sentence length : %d' % (length))
                #print(viterbi_sequence)
            test_pred += predict 
            
            test_batch_hit,test_batch_char = number_of_batch_hits(predict, test_y[slice(i, i + BATCH_SIZE)])            
            print('test batch %d test_batch_hit:%d test_batch_char:%d batch_accuracy:%f' % (i, test_batch_hit, test_batch_char,(test_batch_hit/test_batch_char)*100 ))
            test_hit_all+=test_batch_hit
            test_all_char+=test_batch_char
            
        test_acc = (test_hit_all/test_all_char)*100
        print('test_hit:%d test_all_char:%d accuracy:%f' % (test_hit_all,test_all_char,test_acc ))
        print("--------------------------------------------")
        
        # I'm Saving my model in every epoch for selecting best one in case of overfitting in further epochs
        #if test_acc > 96.20:
        print("accuracy is: %f" % (test_acc))
        save_path = saver.save(sess, root_path+'best_model_path{}/model.ckpt'.format(test_acc))
        print("Model saved in path: %s" % save_path)            
        #add_summary(train_writer, "epoch_test_acc", test_acc, epoch+1)
        
    # Save the variables to disk.
#     save_path = saver.save(sess, root_path+'best_model_path/model.ckpt')
#     print("Model saved in path: %s" % save_path)


start session is ok ....
Train Epoch 1 loss 20.349687
--------------------------------
dev batch 0 loss 4.178298 dev_batch_hit:663 dev_batch_char:679 batch_accuracy:97.643594
dev batch 16 loss 3.464298 dev_batch_hit:681 dev_batch_char:711 batch_accuracy:95.780591
dev batch 32 loss 5.076274 dev_batch_hit:861 dev_batch_char:895 batch_accuracy:96.201117
dev batch 48 loss 2.474782 dev_batch_hit:802 dev_batch_char:807 batch_accuracy:99.380421
dev batch 64 loss 3.413298 dev_batch_hit:744 dev_batch_char:760 batch_accuracy:97.894737
dev batch 80 loss 3.945877 dev_batch_hit:900 dev_batch_char:927 batch_accuracy:97.087379
dev batch 96 loss 3.981677 dev_batch_hit:749 dev_batch_char:774 batch_accuracy:96.770026
dev batch 112 loss 3.372197 dev_batch_hit:671 dev_batch_char:684 batch_accuracy:98.099415
dev batch 128 loss 3.329278 dev_batch_hit:683 dev_batch_char:699 batch_accuracy:97.711016
dev batch 144 loss 2.954602 dev_batch_hit:814 dev_batch_char:826 batch_accuracy:98.547215
dev batch 160 loss 2.

In [0]:
# !tensorboard --logdir logging

***Predicting Section***

In [0]:
#Restore Saved Tensorflow model.

import tensorflow as tf

with tf.Session() as sess:
    #saver = tf.train.Saver()
    
    new_saver = tf.train.import_meta_graph( root_path+'best_model_path/best_model_path97.417_100K_subsetEp5/model.ckpt.meta')
    new_saver.restore(sess, root_path+'best_model_path/best_model_path97.417_100K_subsetEp5/model.ckpt')
    print("Model restored.")
        
   # for tensor in tf.get_default_graph().get_operations():
    #    print(tensor.name)
    
    test_hit_all=0
    test_all_char=0        
    test_pred = []
    for i in range(0, len(test_x), BATCH_SIZE):
        batch_x = test_x[slice(i, i + BATCH_SIZE)]
        batch_x = padding3(batch_x, PAD_ID)

        lengths,unary_scores,transition_param_other = sess.run(
            [seq_length,output,transition_params], feed_dict = {X:batch_x, dropout_keep_prob:1.0})
        predict=[]
        for unary_score,length in zip(unary_scores,lengths):
            if length > 0 :
                viterbi_sequence, _=crf.viterbi_decode(unary_score[:length],transition_param_other)
                predict.append(viterbi_sequence)
            else:
                predict.append("")
            #print('sentence length : %d' % (length))
            #print(viterbi_sequence)
        test_pred += predict 

        test_batch_hit,test_batch_char = number_of_batch_hits(predict, test_y[slice(i, i + BATCH_SIZE)])            
        print('test batch %d test_batch_hit:%d test_batch_char:%d batch_accuracy:%f' % (i, test_batch_hit, test_batch_char,(test_batch_hit/test_batch_char)*100 ))
        test_hit_all+=test_batch_hit
        test_all_char+=test_batch_char

    test_acc = (test_hit_all/test_all_char)*100
    print('test_hit:%d test_all_char:%d accuracy:%f' % (test_hit_all,test_all_char,test_acc ))
    print("--------------------------------------------")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/NLPHW1/best_model_path/best_model_path97.417_100K_subsetEp5/model.ckpt
Model restored.


FailedPreconditionError: ignored

In [0]:
#function for creating BIES file from id's and saving it as utf8 file
def ids_to_tag_file(test_pred,output_file):
    with codecs.open(output_file, 'w','utf-8') as f:
        for i in range(len(test_pred)):
            tag_list = ''
            for j in range(len(test_pred[i])):
                if test_pred[i][j]==0:
                    tag_list +='B'
                elif test_pred[i][j]==1:
                    tag_list+='I'
                elif test_pred[i][j]==2:
                    tag_list+='E'
                elif test_pred[i][j]==3:
                    tag_list+='S'
            f.write(''.join(tag_list).strip())            
            f.write('\n') 
    print("result is ready ...")
   


In [0]:
ids_to_tag_file(test_pred,output_file=root_path+'Datasets/cleaned_data/pku/bies/result.utf8')


result is ready ...
